# EX14 Predicción de texto utilizando redes LSTM

En esta actividad, implementaremos un modelo basado en redes LSTM para la predicción de texto, utilizando una estructura: secuencia a secuencia. Recordemos que una red LSTM es un tipo de red neuronal recurrente. Una red neuronal recurrente es una red neuronal que intenta modelar comportamientos dependientes en el tiempo o secuencia, por ejemplo, el lenguaje, los precios de las acciones y la demanda de electricidad. Para entrenar el modelo, utilizaremos un conjunto de datos (texto) llamado Penn Tree Bank (`PTB`)
[[1](https://catalog.ldc.upenn.edu/LDC99T42)]. 

La actividad se encuentra organizada en las siguientes etapas:

- Preparación de los datos de entrenamiento, pruebas y validación
- Estructura del modelo
- Implementación del modelo
- Compilación y ejecución de modelo
- Predicción

Algunas recomendaciones:
- Realice la actividad en equipo de dos o de manera individual.
- Lea con detenimiento la descripción de cada una de las actividades.
- Consulte la documentación de Keras, para aquellas funciones con las que no esta familiarizado..
- Una vez que termine con el modelo inicial, pruebe con distintos valores de los hyperparámetros.


***¡Iniciemos con la actividad!***



## Librería requeridas

Importemos las librerías que utilizaremos durante la actividad:


In [1]:
#from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np

%tensorflow_version 1.x

Using TensorFlow backend.


In [2]:
# Mount the Google Drive to Google Colab
from google.colab import drive
drive.mount('/content/gdrive/')




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


# 1. Preparación de los conjuntos de datos

Para comenzar, descarguemos de la página del curso, el conjunto de datos `Penn Tree Bank (PTB)`. que utilizaremos como el corpus de entrenamiento y validación.

Un modelo LSTM, requiere como entrada un corpus de palabras únicas asociadas a un indice entero único. Adicionalmente, el corpus necesita ser reconstituido en orden, pero en lugar de palabras de texto se utilizan los identificadores enteros en orden.

Para realizar el procesamiento, implementaremos las siguientes funciones:

- `read_words`, debe dividir un archivo de texto en oraciones (palabras separadas y caracteres, con el finalizador de oración `<eos>`).

- `build_vocabulary`, debe identificar cada palabrá no repetida y asignarle un identificador único (valor entero)

- `words_to_ids` convierte el archivo de texto original en una lista de enteros únicos, donde cada palabra se sustituye con su nuevo identificador entero.

In [0]:
# Función: read_words
def read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", "<eos>").split()

In [0]:
# Función: build_vocabulary
def build_vocabulary(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))
    return word_to_id

In [0]:
def words_to_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

Las tres funciones anteriores nos permitirán preparar los datos originales para construir los datasets de entrenamiento, validación y de prueba, pero con cada palabra representada como un número entero en una lista. Para esto, implementemos la función: `load_data (data_path)` que recibe la ruta en donde se encuentran los archivos:

- ptb_train.txt
- ptb_valid.txt
- ptb_test.txt

La función debe hacer uso de las funciones `read_words`, `build_vocabulary`, `words_to_ids`, para construir el vocabulario, convertir las palabras a identificadores. La función debera retornar:

- `train_data`: datos de entrenamiento de ids (que representan las palabras)
- `valid_data`: datos de validación de ids (que representan las palabras)
- `test_data`: datos de prueba con ids (que representan las palabras)
- `vocabulary`: longitud del vocabulario
- `reversed_dictionary`

In [6]:
def load_data(data_path):
    # Ruta de los archivos de datos
    train_path = os.path.join(data_path, 'ptb_train.txt')
    valid_path = os.path.join(data_path, 'ptb_valid.txt')
    test_path = os.path.join(data_path, 'ptb_test.txt')
    print(train_path)

    #Convertir el texto a una lista de enteros
    word_to_id = build_vocabulary(train_path)
    train_data = words_to_ids(train_path, word_to_id)
    valid_data = words_to_ids(valid_path, word_to_id)
    test_data = words_to_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    # Imprimamos una muestra de los datos procesados
    print(train_data[:5])
    print(" ".join([reversed_dictionary[x] for x in train_data[:5]]))
    
    print(vocabulary)
    #print(word_to_id)
    return train_data, valid_data, test_data, vocabulary, reversed_dictionary


# Ejecutemos la función load_data() para construir los datasets
data_path='./gdrive/My Drive/Colab Notebooks/ptb_corpus'    
train_data, valid_data, test_data, vocabulary, reversed_dictionary = load_data(data_path)

./gdrive/My Drive/Colab Notebooks/ptb_corpus/ptb_train.txt
[9970, 9971, 9972, 9974, 9975]
aer banknote berlitz calloway centrust
10000


## 1.1 Generación de datos (batchs)

Cuando entrenamos redes neuronales, generalmente durante el entrenamiento, alimentamos los datos utilizando `batches`. Keras tiene algunas funciones útiles que pueden extraer datos de entrenamiento de manera automáticamente utilizando un objeto iterador/generador de Python. Para esto necesitamos construir el generador de manera previa e ingresarlo al modelo. En esta actividad, utilizaremos la función de Keras llamada `fit_generator`. Puede consultar la documentación [[aquí](https://keras.io/models/model/#fit_generator)].

El primer argumento para `fit_generator` es la `función iterador` de Python que crearemos, y se usará para extraer lotes de datos durante el proceso de entrenamiento.  Esta función gestionará la extracción de datos, la entrada en el modelo, la ejecución de pasos del gradiente, y el registro de métricas como la precisión. El `iterador` de Python debe tener la siguiente forma:

```Python
while True:
    #do some things to create a batch of data (x, y)
   yield x, y
```

En nuestro caso, implementaremos una clase generadora que contendrá un método que implementa estructura anterior. Para ver ejemplos sobre el uso de generadores y yield puede consultar [[aquí](https://pythontips.com/2013/09/29/the-python-yield-keyword-explained/)].


Los parámetros para la inicialización de la clase son:

- `data`: el primer argumento son los datos que utilizará el generador. Consideremos que los datos pueden ser  entrenamiento, validación o prueba. Así que necesitaremos crear y usar múltiples instancias de la misma clase en las diversas etapas de nuestro ciclo de desarrollo de aprendizaje automático: entrenamiento, validación, pruebas. 

- `num_steps`: este es el número de elementos (time-steps) de la secuencia de entrada en la capa LSTM.

- `batch_size`: se explica por sí mismo.

- `vocabulary`: en nuestro caso es igual a 10,000. 

- `skip_steps`: es la cantidad de palabras que se deben saltar antes de tomar el siguiente dato de entrenamiento del batch.

- `current_idx`: se inicializa en cero. Esta variable es necesaria para realizar un seguimiento de la extracción de datos a través del conjunto de datos completo: una vez que el conjunto de datos se ha consumido en el entrenamiento, debemos restablecer current_idx a cero para que el consumo de datos comience desde el inicio del conjunto de datos nuevamente. En otras palabras, es básicamente un puntero de ubicación de conjunto de datos.

Implementemos el método `__init__(self, data, num_steps, batch_size, vocabulary, skip_step=5)`.

In [0]:
# No ejecute esta celda de código

class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        self.current_idx = 0
        self.skip_step = skip_step

Ahora que tenemos el inicializador, es momento de implementar el método generador que será invocado durante la ejecución de `fit_generator`. Llamaremos al nuestro método `generate()` y debe realizar lo siguiente:

- Crear los arrays para `x` e `y`. 
    - la dimensión de la variable `x` es: (batch_size, num_steps) 
        - el tamaño del batch.
        - la cantidad de palabras en las que vamos a basar nuestras predicciones (longitud de la secuencia)
        
    - la dimensión de la variable `y` es: (batch_size, num_steps, vocabulary)
        - el tamaño del batch
        - la cantidad de palabras en las que vamos a basar nuestras predicciones
        - el tamaño del vocabulario (para esta actividad 10,000)
        
Recordemos que la capa de salida (`y`) de nuestra red LSTM será una capa de salida con una función de activación `softmax`, que asignará una probabilidad a cada una de las 10,000 palabras posibles. La palabra con la mayor probabilidad será la palabra pronosticada; en otras palabras, la red LSTM predecirá una palabra de entre las 10,000 categorías posibles. Por lo tanto, para entrenar la red, necesitamos crear ejemplos de entrenamiento para cada palabra que tenga un 1 en la ubicación de la palabra correcta y ceros en las otras 9,999 ubicaciones (representación one hot vector: [0, 0, 0, ..., 1, 0, ..., 0, 0]). Por lo tanto, para cada palabra objetivo, debe haber un vector de longitud 10,000 con únicamente uno de los elementos del vector establecido en 1.

Ahora, dentro del la estructura repetitiva `while` del generador, acorde a lo visto anteriormente:

```Python
while True:
    #do some things to create a batch of data (x, y)
   yield x, y
```

necesitamos generar un ejemplo de entrenamiento, para cada elemento del `batch`. En caso de que se terminen los datos, es necesario volver a iniciar en 0. Recuerde las dimensiones que tienen las variables `x` e `y`. Adicionalmente, no olvide que `y` deberá tener una representación `One hot vector`. Finalmente, el incremento de `current_idx` deberá ser el valor de `skip_step`.


Una vez finalizado el método `generator()`, como se mencionó anteriormente, podemos configurar instancias de la misma clase para que utilicen los datos de entrenamiento y validación, de la siguiente manera:

```Python
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,skip_step=num_steps)

valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary, skip_step=num_steps)
```

In [0]:
# No ejecute esta celda de código

def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    #Resetear el indice
                    self.current_idx = 0
                    
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # Convertir temp_y en una representación One hot vector
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y

La clase KerasBatchGenerator completa debe quedar como:

In [0]:
# Al completar su código, ejecute esta celda para construir la clase KerasBatchGenerator

class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        self.current_idx = 0
        self.skip_step = skip_step
     
    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    #Resetear el indice
                    self.current_idx = 0
                    
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # Convertir temp_y en una representación One hot vector
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y

Probemos nuestro generador creando dos instancias, una para los datos de entrenamiento y el otro para los datos de validación.

In [0]:
num_steps = 20
batch_size = 30
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)

Ahora que los datos de entrada para nuestra modelo están configurados y listos, es hora de crear la red LSTM.

## 2. Estructura del modelo

<img src="LSTM-model.png" style="width:900;height:600px;">

<caption><center> Figura 1: Modelo de predicción de texto (estructura: secuencia a secuencia) </center></caption>

La figura 1 describe la propuesta de la estructura inicial para el modelo. Veamos cada uno de los elementos de la estructura:

### 2.1 Capa word embedding

Recordemos que la entrada a una red LSTM no son escalares de valor único, sino secuencias (vectores de cierta longitud). Del mismo modo, todos los los pesos y bias son: matrices y vectores respectivamente. Ahora, acorde a lo visto en las sesiones anteriores ¿cómo representamos palabras para ingresarlas a una red neuronal?, la respuesta son los word embeddings. El word embedding implica tomar una palabra y encontrar una representación vectorial de esa palabra que capture algún significado semántico de la misma.  En el algoritmo `Word2Vec`, el de la palabra se cuantifica por el contexto, es decir por aquellas palabras que aparecen en oraciones cercanas a las mismas palabras.

Los vectores de palabras, se pueden aprender por separado o se pueden aprender durante el entrenamiento de la red LSTM. En esta actividad, configuraremos lo que se llama una capa embedding, para convertir cada palabra en un word embedding. Para construir un capa Embedding y agregarla al modelo en Keras, utilizaremos el siguiente método:

`model.add(Embedding( parameters ))`

Algunos de los parámetros que requiere la capa Embedding, y que utilizaremos en esta actividad son:

- input_dim:       Tamaño del vocabulario
- input_length:    Longitud de la secuencia
- output_dim:      Dimensión del embedding vector

Para ver la documentación de la capa Embedding consulte [[aquí](https://keras.io/layers/embeddings/#embedding)].

Para el diseño e implementación de redes LSTM, considere que para la capa Embedding:

- La dimensión de la entrada es:

    - Tensores 2D: (batch_size, sequence_length).

- La dimensión de la salida es:

    - Tensores 3D: (batch_size, sequence_length, output_dim).


***Nota: la capa Embedding únicamente puede utilizarse como la mapa inicial del modelo.***

La salida de la capa embedding, tendrá las dimensión: (20, 30, 500).

### 2.2 Capa LSTM


Los datos de salida de la capa Embedding, son la entrada en dos capas "apiladas" de celdas LSTM (de un tamaño oculto de longitud 500). En el diagrama de la figura 1, la red LSTM se muestra desenrollada. La salida de estas celdas desenrolladas tienen una dimensión: (tamaño de lote, número de pasos de tiempo, tamaño oculto).

Por lo general, debe coincidir el tamaño de la salida de la capa Emedding con el número de capas ocultas en la celda LSTM. Tal vez se pregunte de dónde provienen las capas ocultas en la celda LSTM. En sesiones anteriores se presentarion las celdas LSTM de manera abstracta, y simplemente se mostró el flujo de los datos y las operaciones que se realizan sobre ellos. Sin embargo, cada unidad de activación (por ejemplo, `sigmoide` y `tanh`) en la celda, es en realidad un conjunto de unidades cuyo número es igual al tamaño de la capa oculta. Por lo tanto, cada uno de las "unidades" en la celda LSTM es en realidad un grupo de unidades de redes neuronal, como en cada capa de una red neuronal `FC`. Para agregar a nuestro modelo una capa LSTM utilizando Keras, utilizaremos:

`model.add(LSTM( parameters ))`

Algunos de los parámetros que requiere la capa LSTM, y que utilizaremos en esta actividad son:

- units (int > 0). Dimensión de la salida.

- return_sequences: True, si queremos que retorne toda la secuencia. False, si queremos la última salida de la sequencia.

Puede ver la documentación de la capa LSTM consulte [[aquí](https://keras.io/layers/recurrent/#lstm)].

### 2.3 Capa TimeDistributed

Los datos de salida de la segunda capa LSTM, son la entrada a una capa de Keras llamada `TimeDistribute`. Esta capa agrega una capa independiente para cada paso de tiempo en el modelo recurrente. Entonces, por ejemplo, si tenemos 10 time-step en un modelo (es decir, trabaja con secuencias de longitud 10), una capa `TimeDistributed` que opera en una capa `Dense` produciría 10 capas densas independientes, una para cada time-step. La activación para estas capas densas está configurada para ser softmax en la capa final de nuestro modelo Keras LSTM. Para agregar a nuestro modelo una capa `TimeDistributed` con Keras, utilizaremos:

`keras.layers.TimeDistributed(layer)`

En nuestro caso, `layer` será una capa `Dense`. Veamos un ejemplo del uso de `TimeDistributed` para aplicar una capa densa a cada uno de 10 time-steps, de manera independientemente:

```python
model.add(TimeDistributed(Dense(8), input_shape=(10, 16)))     # model.output_shape == (None, 10, 8)
```

Finalmente, la capa de salida tiene una activación softmax, considerando para la predicción del texto que nuestro vocabulario tiene una longitud de 10,000 palabras (clases).

## 3. Implementación del modelo LSTM en Keras


Para la implementación del modelo utilizaremos el método secuencial. Básicamente, la metodología secuencial permite apilar fácilmente capas en la red LSTM sin preocuparnos demasiado por todos los tensores (y sus dimensiones) que fluyen a través del modelo.

Tomemos como base el driagrama de la figura 1 para implementar la red LSTM:

In [11]:
hidden_size = 500
use_dropout = True

model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 4. Compilación y ejecución del modelo LSTM

Una vez que hemos completado el modelo, ejecutemos el método de compilación. En este método, se debe especificar el tipo de pérdida que Keras debería usar para entrenar el modelo. En este caso, utilizaremos `categorical_crossentropy`. El optimizador que se utilizaremos es `Adam`. Finalmente, la métrica que utilizaremos es: `categorical_accuracy`, que nos permite ver cómo mejora la precisión durante el entrenamiento.

Adicionalmente, utilizaremos un callback para un `ModelCheckPoint` para guardar el modelo después de cada época, lo que puede ser útil para cuando se está realizando un entrenamiento largo.

In [12]:
# Entrenamiento del modelo

optimizer = Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
print(model.summary())

#Model check pointer para almacenar el modelo cada epoca
checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=1)

#Número de epocas de entrenamiento
num_epochs = 1

model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                    validation_data=valid_data_generator.generate(),
                    validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[checkpointer])
model.save(data_path + "final_model.hdf5")



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 500)           5000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 500)           2002000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 500)           2002000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 500)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 10000)         5010000   
_________________________________________________________________
activation_1 (Activation)    (None, 20, 10000)         0         
Total params: 14,014,000
Trainable params: 14,014,000
Non-trainable params: 0
________________________________________

## 5. Uso del modelo para realizar predicciones

In [13]:
# Utilicemos el modelo entrenado para realizar algunas predicciones

model = load_model(data_path + "/model-01.hdf5")
dummy_iters = 40
example_training_generator = KerasBatchGenerator(train_data, num_steps, 1, vocabulary,
                                                     skip_step=1)
print("Training data:")
    
for i in range(dummy_iters):
    dummy = next(example_training_generator.generate())
    
num_predict = 10
true_print_out = "Actual words: "
pred_print_out = "Predicted words: "
for i in range(num_predict):
    data = next(example_training_generator.generate())
    prediction = model.predict(data[0])
    predict_word = np.argmax(prediction[:, num_steps-1, :])
    true_print_out += reversed_dictionary[train_data[num_steps + dummy_iters + i]] + " "
    pred_print_out += reversed_dictionary[predict_word] + " "
    
print(true_print_out)
print(pred_print_out)

# test data set
dummy_iters = 40
example_test_generator = KerasBatchGenerator(test_data, num_steps, 1, vocabulary,
                                             skip_step=1)
print("Test data:")
for i in range(dummy_iters):
    dummy = next(example_test_generator.generate())
num_predict = 10
true_print_out = "Actual words: "
pred_print_out = "Predicted words: "
for i in range(num_predict):
    data = next(example_test_generator.generate())
    prediction = model.predict(data[0])
    predict_word = np.argmax(prediction[:, num_steps - 1, :])
    true_print_out += reversed_dictionary[test_data[num_steps + dummy_iters + i]] + " "
    pred_print_out += reversed_dictionary[predict_word] + " "
    
print(true_print_out)
print(pred_print_out)

Training data:
Actual words: chairman of consolidated gold fields plc was named a nonexecutive 
Predicted words: <unk> of the the <eos> <eos> <eos> the <eos> <unk> 
Test data:
Actual words: unable to cool the selling panic in both stocks and 
Predicted words: the to the the <unk> of <eos> the <unk> <eos> 
